# Azure Blob Storage Demo

## Preparation

In [22]:
#r "nuget: Azure.Storage.Blobs"

Installed Packages Azure.Storage.Blobs, 12.25.0

Import namespaces:

In [23]:
using System.IO;
using Azure.Storage.Blobs;
using Azure.Storage.Blobs.Models;

Set up connection string:

In [ ]:
string connectionString = "DefaultEndpointsProtocol=https;AccountName=ztechtest;AccountKey=XXXX;EndpointSuffix=core.windows.net";

## Upload and download files

Upload a file to Azure Blob Storage:

In [25]:
// Set your container and blob names
string containerName = "my-container";
string blobName = "sample.txt";

// Create a sample file
string localFilePath = Path.GetTempFileName();
System.IO.File.WriteAllText(localFilePath, "Hello, World!");

// Create a BlobServiceClient
var blobServiceClient = new BlobServiceClient(connectionString);

// Get a reference to a container
var containerClient = blobServiceClient.GetBlobContainerClient(containerName);
await containerClient.CreateIfNotExistsAsync();

// Get a reference to a blob
var blobClient = containerClient.GetBlobClient(blobName);

// Upload the file
using (var fileStream = System.IO.File.OpenRead(localFilePath)) {
    await blobClient.UploadAsync(fileStream, overwrite: true);
}

Console.WriteLine($"Uploaded file {localFilePath} to {blobClient.Uri}.");

List blobs in a container:

In [26]:
// List blobs in the container
Console.WriteLine($"Blobs in container '{containerName}':");
await foreach (var blobItem in containerClient.GetBlobsAsync()) {
    var fileSize = blobItem.Properties.ContentLength ?? 0;
    Console.WriteLine($"- {blobItem.Name} ({fileSize} bytes)");
}

Download a blob:

In [27]:
// Download the blob
string downloadFilePath = Path.GetTempFileName();

using (var downloadStream = File.OpenWrite(downloadFilePath)) {
    await blobClient.DownloadToAsync(downloadStream);
}

Console.WriteLine($"Downloaded {blobClient.Uri} to {downloadFilePath}.");

## Working with blob metadata

Upload files `*.jpg` from `C:\Windows\Web\Wallpaper\ThemeA` to container:

In [28]:
var directoryPath = @"C:\Windows\Web\Wallpaper\ThemeA";
var jpgFiles = Directory.GetFiles(directoryPath, "*.jpg");
foreach (var filePath in jpgFiles) {
    var fileName = Path.GetFileName(filePath);
    var fileBlobClient = containerClient.GetBlobClient(fileName);
    byte[] fileBytes = File.ReadAllBytes(filePath);
    byte[] md5Bytes;
    string md5Hash;
    using (var md5 = System.Security.Cryptography.MD5.Create())
    {
        md5Bytes = md5.ComputeHash(fileBytes);
        md5Hash = BitConverter.ToString(md5Bytes).Replace("-", string.Empty).ToLowerInvariant();
    }
    var metadata = new Dictionary<string, string>
    {
        { "OriginalLocation", filePath }
    };
    var headers = new BlobHttpHeaders { 
        ContentType = "image/jpeg", 
        ContentHash = md5Bytes 
    };
    using (var fileStream = new MemoryStream(fileBytes))
    {
        await fileBlobClient.UploadAsync(fileStream, new BlobUploadOptions { HttpHeaders = headers, Metadata = metadata });
        Console.WriteLine($"Uploaded {fileName} with metadata and Content-MD5 to container {containerName}");
    }
}

Uploaded img21.jpg with metadata and Content-MD5 to container my-container


In [29]:
await foreach (var blobItem in containerClient.GetBlobsAsync())
{
    var fileBlobClient = containerClient.GetBlobClient(blobItem.Name);
    Console.WriteLine($"Blob: {blobItem.Name}");

    var properties = await fileBlobClient.GetPropertiesAsync();
    Console.WriteLine($"  Size: {properties.Value.ContentLength} bytes");
    Console.WriteLine($"  Content Type: {properties.Value.ContentType}");
    Console.WriteLine($"  Last Modified: {properties.Value.LastModified}");

    if(properties.Value.ContentHash != null)
    {
        Console.WriteLine($"  Content-MD5: {BitConverter.ToString(properties.Value.ContentHash).Replace("-", string.Empty).ToLowerInvariant()}");
    }
    else
    {
        Console.WriteLine("  Content-MD5: Not available");
    }
    if(properties.Value.Metadata.Any()) {
        Console.WriteLine($"  Metadata:");
        foreach (var kvp in properties.Value.Metadata)
        {
            Console.WriteLine($"    {kvp.Key}: {kvp.Value}");
        }
    }
    Console.WriteLine();
}

  Content Type: image/jpeg
  Last Modified: 8/13/2025 5:49:00 PM +00:00
  Content-MD5: 3f1becb6b934f6cae6c560d183e804f0
    OriginalLocation: C:\Windows\Web\Wallpaper\ThemeA\img20.jpg

Blob: img21.jpg
  Size: 833851 bytes
  Content Type: image/jpeg
  Last Modified: 8/13/2025 5:49:01 PM +00:00
  Content-MD5: e3a4a7f46f916db794816be4e60bff97
  Metadata:
    OriginalLocation: C:\Windows\Web\Wallpaper\ThemeA\img21.jpg

Blob: img22.jpg
  Size: 760748 bytes
  Content Type: image/jpeg
  Last Modified: 8/13/2025 5:49:01 PM +00:00
  Content-MD5: cd169f22e8b5eae155121439f0b8be2e
  Metadata:
    OriginalLocation: C:\Windows\Web\Wallpaper\ThemeA\img22.jpg

Blob: img23.jpg
  Size: 907112 bytes
  Content Type: image/jpeg
  Last Modified: 8/13/2025 5:49:02 PM +00:00
  Content-MD5: dc2b1fd8e198ea0225b095456c7f4518
  Metadata:
    OriginalLocation: C:\Windows\Web\Wallpaper\ThemeA\img23.jpg

Blob: sample.txt
  Size: 13 bytes
  Content Type: application/octet-stream
  Content-MD5: 65a8e27d8879283831b664b

## Cleanup

In [30]:
await containerClient.DeleteIfExistsAsync();
Console.WriteLine($"Deleted container: {containerName}");

Deleted container: my-container
